In [1]:
from selenium import webdriver
import time
import threading
from utils import amountToBase, showCycle, percent
from configs import logging, MIN_PERCENT_REQUIRED, BRL_AMOUNT_TRADE, UPDATE_TICK_RATE, LOGIN_TOKEN
from datetime import datetime as dt
from playsound import playsound
from classes.Robots import WebRobot, BiscointRobot

In [2]:
# Initial configs

# Selenium
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)

driver.get("https://biscoint.io")
driver.execute_script(f"window.localStorage.setItem('Meteor.loginToken','{LOGIN_TOKEN}');")
driver.get("https://biscoint.io/dashboard/portfolio")

# Web Robot
web_robot = WebRobot(driver)
biscoint_robot = BiscointRobot()

In [3]:
def updateTick(cycle_count):
    global ticker
    global amount_btc_to_trade
    if cycle_count % UPDATE_TICK_RATE == 0:
        try:
            ticker = bsc.get_ticker()
            amount_btc_to_trade =  amountToBase(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])
        except Exception as e:
            logging.error(f"Error on updating tick {e}")

# if spread is high, sleep, else speed up checks
def waitForNextCycle(calculated_percent):
    if calculated_percent < -0.1:
        time.sleep(sleep_time_offers)
    else:
        time.sleep(0.05)

def async_offer(robot, op:str,amount:str,isQuote:bool):
    global request_orders
    response = None
    try:
        response = robot.get_offer(op,amount,isQuote) 
        request_orders[op] = response
    except Exception as e:
        print(response)
        print("Erro ao rodar thread",e)


In [4]:
#Return initial balance from Biscoint
initial_balance = last_balance = web_robot.api.get_balance()
# Calculate the rate limit of request to Biscoint API
endpoints_meta = web_robot.api.get_meta()
rate_limit_offer = endpoints_meta['endpoints']['offer']['post']['rateLimit']
sleep_time_offers = ((rate_limit_offer["windowMs"] / rate_limit_offer["maxRequests"]) / 1000) * 1.5
# Convert the BRL amount of trading to BTC
ticker = web_robot.api.get_ticker()

amount_btc_to_trade = amountToBase(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])

percent_record = -1 # Will refresh every time when a new positive spread is achived

logging.info(f"Initial balance: {initial_balance}")
# Arbitrage Cycle
cycle_count = 1


NameError: name 'bsc' is not defined

In [6]:
assert async_offer
while True:
    try:
        start_time = dt.now()

        # Get Offers in thread
        request_orders = {}
        thread_buy = threading.Thread(target=async_offer, args=(web_robot, 'buy',str(amount_btc_to_trade), False))
        thread_sell = threading.Thread(target=async_offer, args=(biscoint_robot,'sell',str(amount_btc_to_trade), False))
        thread_buy.start()
        thread_sell.start()
        thread_buy.join()
        thread_sell.join()

        # Get buy and sell offers of this cycle
        #buy = web_robot.get_offer(op='buy',amount=str(amount_btc_to_trade),is_quote=False)
        #sell = web_robot.get_offer(op='sell',amount=str(amount_btc_to_trade),is_quote=False)
        buy = request_orders['buy']
        sell = request_orders['sell']

        # Calculate if arbitrage is possible
        calculated_percent = percent(buy['efPrice'],sell['efPrice'])

        showCycle(cycle_count,calculated_percent)
        
        if calculated_percent > percent_record:
            logging.info(f"Percent Record Reached!! : {calculated_percent} at {dt.now()}")
            percent_record = calculated_percent
        # If Arbitrage is possible, confirm offers
        if calculated_percent >= MIN_PERCENT_REQUIRED:
            logging.info(f"Arbitrage oportunity: buy:{buy['efPrice']}   sell:{sell['efPrice']}")
            playsound('beep.wav')
            
            #Execute orders
            if float(last_balance['BRL']) < float(BRL_AMOUNT_TRADE):
                executed_sell = web_robot.confirm_offer(sell['offerId'])
                executed_buy = web_robot.confirm_offer(buy['offerId'])
            else:
                executed_buy = web_robot.confirm_offer(buy['offerId'])
                executed_sell = web_robot.confirm_offer(sell['offerId'])

            last_balance = bsc.get_balance()
            logging.info(f"New Balance is: {last_balance}")

            break
            

        end_time = dt.now()
        seconds_elapsed = (end_time - start_time).total_seconds()
        logging.debug(f"Cycle took {seconds_elapsed} seconds")
        cycle_count +=1

        updateTick(cycle_count)
        waitForNextCycle(calculated_percent)
    except Exception as e:
        logging.error(e)

Percent Record Reached!! : -0.506 at 2021-10-11 00:50:19.020204
Percent Record Reached!! : -0.343 at 2021-10-11 00:50:29.285650
Percent Record Reached!! : -0.307 at 2021-10-11 00:50:41.889890
Percent Record Reached!! : -0.289 at 2021-10-11 00:51:20.958450
Percent Record Reached!! : -0.163 at 2021-10-11 00:51:31.718617
Cycle 100 | Last percent : -0.954 |
Cycle 200 | Last percent : -0.9 |
Cycle 300 | Last percent : -0.865 |
Cycle 400 | Last percent : -0.974 |
Cycle 500 | Last percent : -0.92 |
Cycle 600 | Last percent : -0.99 |
Cycle 700 | Last percent : -0.846 |
Cycle 800 | Last percent : -0.865 |
Cycle 900 | Last percent : -0.775 |
Cycle 1000 | Last percent : -0.81 |
Cycle 1100 | Last percent : -0.791 |
Cycle 1200 | Last percent : -0.684 |
Cycle 1300 | Last percent : -1.15 |
Cycle 1400 | Last percent : -0.937 |
Cycle 1500 | Last percent : -0.668 |


In [10]:
driver.quit()